In [30]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [35]:
df = pd.read_excel('datos_julio.xlsx') 
# df2 = pd.read_csv('datos.csv', delimiter=';')

In [11]:
# Unir los dos Dataframe del 20323 y 2024 
# ESTE PASO EL OPCIONAL Solo si cargan los datos df2
# columnasEnComun = ['Temperatura de módulo | Sensor Card / Box (1)','Irradiación | Sensor Card / Box (1)','Temperatura ambiente | Sensor Card / Box (1)','Fecha y hora']
# df = pd.concat([df[columnasEnComun], df2[columnasEnComun]])
# df

,Temperatura de módulo | Sensor Card / Box (1),Irradiación | Sensor Card / Box (1),Temperatura ambiente | Sensor Card / Box (1),Fecha y hora
0,"31,00","288,00","24,00",01.03.2024 15:45
1,"34,00","293,62","24,00",01.03.2024 15:50
2,"34,00","340,00","24,00",01.03.2024 15:55
3,"36,00","469,00","24,00",01.03.2024 16:00
4,"36,00","385,00","24,00",01.03.2024 16:05
...,...,...,...,...
57881,NaN,NaN,NaN,NaN
57882,NaN,NaN,NaN,NaN
57883,NaN,NaN,NaN,NaN
57884,NaN,NaN,NaN,NaN


In [36]:
df = df.loc[1:]
# Renombrar columnas
df.rename(columns={'Temperatura de módulo | Sensor Card / Box (1)': 'temperaturaSensor'}, inplace=True)
df.rename(columns={'Temperatura ambiente | Sensor Card / Box (1)': 'temperaturaAmbiente'}, inplace=True)
df.rename(columns={'Irradiación | Sensor Card / Box (1)': 'irradiacion'}, inplace=True)
df.rename(columns={'Fecha y hora': 'fecha'}, inplace=True)

# Quedarme con dos columnas de entrada y una de salida(irradiacion)
df = df[['fecha','temperaturaAmbiente','temperaturaSensor','irradiacion']]

# Eliminar los valores nulos
df.dropna(inplace=True)

df['irradiacion'] = df['irradiacion'].apply(lambda x: float(str(x).strip().replace(',','.')))

# Solo quedarme con los valores que hay irradiación (Día)
df = df[df['irradiacion'] > 0]
df


,fecha,temperaturaAmbiente,temperaturaSensor,irradiacion
190,01.03.2024 15:45,24,31,288.00
191,01.03.2024 15:50,24,34,293.62
192,01.03.2024 15:55,24,34,340.00
193,01.03.2024 16:00,24,36,469.00
194,01.03.2024 16:05,24,36,385.00
...,...,...,...,...
35810,31.07.2024 18:10,19,17,19.00
35811,31.07.2024 18:15,18.99,16.99,15.94
35812,31.07.2024 18:20,18,16,10.00
35813,31.07.2024 18:25,18,16,4.98


In [37]:
# Transformar la fecha a datetime
df['fecha'] = pd.to_datetime(df['fecha'], format='%d.%m.%Y %H:%M')
df['hora'] = [i.hour for i in df['fecha']]
df['minuto'] = [i.minute for i in df['fecha']]
# Parsear las columnas temperaturas a float
df['temperaturaAmbiente'] = df['temperaturaAmbiente'].apply(lambda x: float(str(x).strip().replace(',','.')))
df['temperaturaSensor'] = df['temperaturaSensor'].apply(lambda x: float(str(x).strip().replace(',','.')))
df['tiempo'] = df.apply(lambda row: row['hora'] + (row['minuto'] // 5) * 0.083, axis=1)

### Separar los datos para Entrenamiento y TEST

In [40]:
X_modulo = df[['temperaturaAmbiente', 'tiempo']].values
y_modulo = df['temperaturaSensor'].values

X_irradiacion = df[['temperaturaSensor', 'tiempo']].values
y_irradiacion = df['irradiacion'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
# Dividir los datos en conjuntos de entrenamiento y prueba
X_modulo_train, X_modulo_test, y_modulo_train, y_modulo_test = train_test_split(X_modulo, y_modulo, test_size=0.2, random_state=42)
X_irradiacion_train, X_irradiacion_test, y_irradiacion_train, y_irradiacion_test = train_test_split(X_irradiacion, y_irradiacion, test_size=0.2, random_state=42)
# Definir la arquitectura de la RNA
model_modulo = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(2,)), #  variables de entrada
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Capa de salida con una sola neurona para predecir la radiación
])
model_irradiacion = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(2,)), #  variables de entrada
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Capa de salida con una sola neurona para predecir la radiación
])


In [41]:
# Compilar el modelo
model_modulo.compile(optimizer='adam', loss='mean_squared_error')
model_irradiacion.compile(optimizer='adam', loss='mean_squared_error')

In [42]:
# Entrenar el modelo
model_modulo.fit(X_modulo_train, y_modulo_train, epochs=50, batch_size=32, validation_split=0.2)
model_irradiacion.fit(X_irradiacion_train, y_irradiacion_train, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50
361/361 [==============================] - 1s 2ms/step - loss: 150.3956 - val_loss: 44.5689
Epoch 2/50
361/361 [==============================] - 1s 2ms/step - loss: 40.4857 - val_loss: 39.5610
Epoch 3/50
361/361 [==============================] - 1s 2ms/step - loss: 37.9690 - val_loss: 36.7953
Epoch 4/50
361/361 [==============================] - 1s 2ms/step - loss: 34.9477 - val_loss: 32.7060
Epoch 5/50
361/361 [==============================] - 1s 2ms/step - loss: 30.5597 - val_loss: 28.3202
Epoch 6/50
361/361 [==============================] - 1s 2ms/step - loss: 26.4448 - val_loss: 24.9830
Epoch 7/50
361/361 [==============================] - 1s 2ms/step - loss: 23.0475 - val_loss: 21.7761
Epoch 8/50
361/361 [==============================] - 1s 2ms/step - loss: 21.0821 - val_loss: 20.1129
Epoch 9/50
361/361 [==============================] - 1s 2ms/step - loss: 19.8186 - val_loss: 18.7407
Epoch 10/50
361/361 [==============================] - 1s 2ms/step - loss: 19.162

In [43]:
from sklearn.metrics import mean_squared_error, r2_score

y_modulo_pred = model_modulo.predict(X_modulo_test)
y_irradiacion_pred = model_irradiacion.predict(X_irradiacion_test)

mse1 = mean_squared_error(y_modulo_test, y_modulo_pred)
mse2 = mean_squared_error(y_irradiacion_test, y_irradiacion_pred)

r2_modulo = r2_score(y_modulo_test, y_modulo_pred)
r2_irradiacion = r2_score(y_irradiacion_test, y_irradiacion_pred)

print("Coeficiente de determinación modulo (R²):", r2_modulo)
print("Coeficiente de determinación iradiacion (R²):", r2_irradiacion)

113/113 [==============================] - 0s 1ms/step
Coeficiente de determinación modulo (R²): 0.8476715948331918
Coeficiente de determinación iradiacion (R²): 0.8257024652495513


In [44]:
# Guardar modelo
model_modulo.save('modelo_modulo_rna_v3')
model_irradiacion.save('modelo_irradiacion_rna_v3')

INFO:tensorflow:Assets written to: modelo_modulo_rna_v3\assets


INFO:tensorflow:Assets written to: modelo_modulo_rna_v3\assets


INFO:tensorflow:Assets written to: modelo_irradiacion_rna_v3\assets


INFO:tensorflow:Assets written to: modelo_irradiacion_rna_v3\assets
